In [1]:
import segyio
import numpy as np
import glob
import statistics

import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)

ModuleNotFoundError: No module named 'torch'

# Training data processing

In [3]:
path1 = r"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/seismic_train"
path2 = r"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/fault_train"
all_seis = sorted(glob.glob(path1+"/*.npy"))
all_faults = sorted(glob.glob(path2+"/*.npy"))

In [4]:
all_seis[5]

"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/seismic_train\\seistrain6.npy"

In [5]:
all_faults[4]

"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/fault_train\\faulttrain5.npy"

In [6]:
#seismic data with new shape: images tranformed into 1024*512 shape
train_data = np.empty((0,512,512),dtype='float32')
for filename in all_seis:
    seis = np.load(filename)
    seis_ns = seis[:,:2048,750:1262]          
    seis_ns1, seis_ns2, seis_ns3, seis_ns4 = np.split(seis_ns, 4, axis = 1)
    seis_ns_final = np.concatenate((seis_ns1,seis_ns2,seis_ns3,seis_ns4),axis=0)
    train_data = np.append(train_data,seis_ns_final,axis=0)
train_data.shape

(2400, 512, 512)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(train_data[0,:,:].T,cmap='gray',  vmin=0, vmax=0.4, alpha = 0.5)

In [ ]:
#fault data with new shape: images tranformed into 512*512 shape
fault_data = np.empty((0,512,512),dtype='float32')
for filename in all_faults:
    fault = np.load(filename)
    fault_ns = fault[:,:2048,750:1262]          
    fault_ns1, fault_ns2, fault_ns3, fault_ns4 = np.split(fault_ns, 4, axis = 1)
    fault_ns_final = np.concatenate((fault_ns1,fault_ns2,fault_ns3,fault_ns4),axis=0)
    fault_data = np.append(fault_data,fault_ns_final,axis=0)
fault_data.shape

In [ ]:
threshold = 0.025 * 512 * 512
fault_data_refined = np.empty((0,512,512),dtype='float32')
train_data_refined = np.empty((0,512,512),dtype='float32')
for i in range(2400):
    if np.count_nonzero(fault_data[i] == 1) > threshold:
        fault_data_refined=np.append(fault_data_refined,fault_data[i:i+1],axis=0)
        train_data_refined=np.append(train_data_refined,train_data[i:i+1],axis=0)
fault_data_refined.shape , train_data_refined.shape

In [ ]:
train_seis_img = torch.from_numpy(train_data_refined).to(torch.float32)
print(train_seis_img.shape)
print(train_seis_img.dtype)

In [ ]:
fig = plt.figure(figsize=(20,20))
plt.imshow(train_seis_img[0,:,:].T,cmap='gray',  vmin=0, vmax=0.4, alpha = 0.5)

In [ ]:
train_fault_img = torch.from_numpy(fault_data_refined).to(torch.float32)
print(train_fault_img.shape)
print(train_fault_img.dtype)

In [ ]:
train_seis_list = []
train_fault_list = []
for i in range(1512):
    train_seis_list.append(train_seis_img[i].unsqueeze(0))
    train_fault_list.append(train_fault_img[i].unsqueeze(0))

In [ ]:
train_seis_list[0].shape

In [ ]:
# Loading data into Pytorch Dataset Utitlity
train_volumes = torch.stack(train_seis_list)
train_labels = torch.stack(train_fault_list)
train_dataset = torch.utils.data.TensorDataset(train_volumes, train_labels)

In [ ]:
train_volumes.shape, train_labels.shape, train_volumes.dtype

In [ ]:
# One last sanity check
fig = plt.figure(figsize=(40,40))
    
ax = fig.add_subplot(331)    
plt.imshow(train_volumes[0].T, cmap="gray");
ax.set_title("Seismic Image")
    
ax = fig.add_subplot(332)
ax.imshow(train_volumes[0].T, cmap = 'gray')
ax.imshow(train_labels[0].T, cmap = 'gray', vmin=0, vmax=1, alpha=0.3)
ax.set_title("Fault")
plt.show();

# Validation data processing

In [ ]:
path3 = r"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/seismic_validation"
path4 = r"D:/Earth and Energy Sciences class/Spring'2022/CSCE 588- Neural Networks/CNN project/All data/Fault_detection_gigabyte_data/fault_validation"
valid_seis = sorted(glob.glob(path3+"/*.npy"))
valid_faults = sorted(glob.glob(path4+"/*.npy"))

In [ ]:
valid_seis[0]

In [ ]:
valid_faults[0]

In [ ]:
#fault data with new shape: images tranformed into 512*512 shape
valid_seis_data = np.empty((0,512,512),dtype='float32')
for filename in valid_seis:
    seis_v = np.load(filename)
    seis_ns = seis_v[:,250:2298,800:1312]          
    seis_ns1, seis_ns2, seis_ns3, seis_ns4 = np.split(seis_ns, 4, axis = 1)
    seis_ns_final = np.concatenate((seis_ns1,seis_ns2,seis_ns3,seis_ns4),axis=0)
    valid_seis_data = np.append(valid_seis_data,seis_ns_final,axis=0)
valid_seis_data.shape

In [ ]:
#fault data with new shape: images tranformed into 512*512 shape
valid_fault_data = np.empty((0,512,512),dtype='float32')
for filename in valid_faults:
    fault_v = np.load(filename)
    fault_ns = fault_v[:,250:2298,800:1312]          
    fault_ns1, fault_ns2, fault_ns3, fault_ns4 = np.split(fault_ns, 4, axis = 1)
    fault_ns_final = np.concatenate((fault_ns1,fault_ns2,fault_ns3,fault_ns4),axis=0)
    valid_fault_data = np.append(valid_fault_data,fault_ns_final,axis=0)
valid_fault_data.shape

In [ ]:
threshold = 0.025 * 512 * 512
valid_fault_data_refined = np.empty((0,512,512),dtype='float32')
valid_train_data_refined = np.empty((0,512,512),dtype='float32')
for i in range(400):
    if np.count_nonzero(valid_fault_data[i] == 1) > threshold:
        valid_fault_data_refined=np.append(valid_fault_data_refined,valid_fault_data[i:i+1],axis=0)
        valid_train_data_refined=np.append(valid_train_data_refined,valid_seis_data[i:i+1],axis=0)
valid_fault_data_refined.shape , valid_train_data_refined.shape

In [ ]:
valid_seis_img = torch.from_numpy(valid_train_data_refined).to(torch.float32)
print(valid_seis_img.shape)
print(valid_seis_img.dtype)

In [ ]:
valid_fault_img = torch.from_numpy(valid_fault_data_refined).to(torch.float32)
print(valid_fault_img.shape)
print(valid_fault_img.dtype)

In [ ]:
valid_seis_list = []
valid_fault_list = []
for i in range(400):
    valid_seis_list.append(valid_seis_img[i].unsqueeze(0))
    valid_fault_list.append(valid_fault_img[i].unsqueeze(0))

In [ ]:
valid_seis_list[0].shape

In [ ]:
# Loading data into Pytorch Dataset Utitlity
valid_volumes = torch.stack(valid_seis_list)
valid_labels = torch.stack(valid_fault_list)
valid_dataset = torch.utils.data.TensorDataset(valid_volumes, valid_labels)

In [ ]:
valid_volumes.shape, valid_labels.shape, valid_volumes.dtype

In [ ]:
# One last sanity check
fig = plt.figure(figsize=(40,40))
    
ax = fig.add_subplot(331)    
plt.imshow(valid_volumes[0].T, cmap="gray");
ax.set_title("Seismic Image")
    
ax = fig.add_subplot(332)
ax.imshow(valid_volumes[0].T, cmap = 'gray')
ax.imshow(valid_labels[0].T, cmap = 'gray', vmin=0, vmax=1, alpha=0.3)
ax.set_title("Fault")
plt.show();

# Build the CNN

In [ ]:
class ContractingBlock(nn.Module):
    '''
    ContractingBlock Class
    Performs two convolutions followed by a max pool operation.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels):
        super(ContractingBlock, self).__init__()
        
        # double the number of channels in the first convolution
        # and keep the same number of channels in the second.
        
        self.conv1 = nn.Conv2d(input_channels, 2*input_channels, kernel_size=3, padding=(1,1))
        self.conv2 = nn.Conv2d(2*input_channels, 2*input_channels, kernel_size=3, padding=(1,1))
        self.activation = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        '''
        Function for completing a forward pass of ContractingBlock: 
        Given an image tensor, completes a contracting block and returns the transformed tensor.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        x = self.conv1(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.maxpool(x)
        return x

In [ ]:
class ExpandingBlock(nn.Module):
    '''
    ExpandingBlock Class
    Performs an upsampling, a convolution, a concatenation of its two inputs,
    followed by two more convolutions.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels):
        super(ExpandingBlock, self).__init__()
        
        # "Every step in the expanding path consists of an upsampling of the feature map"
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)      
        self.conv1 = nn.Conv2d(input_channels, input_channels//2, kernel_size=3, padding=(1,1))
        self.conv2 = nn.Conv2d(input_channels, input_channels//2, kernel_size=3, padding=(1,1))
        self.conv3 = nn.Conv2d(input_channels//2, input_channels//2, kernel_size=3, padding=(1,1))
        
        self.activation = nn.ReLU() # "each followed by a ReLU"
 
    def forward(self, x, skip_con_x):
        '''
        Function for completing a forward pass of ExpandingBlock: 
        Given an image tensor, completes an expanding block and returns the transformed tensor.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
            skip_con_x: the image tensor from the contracting path (from the opposing block of x)
                    for the skip connection
                    
        Note: In the original Unet implementation, the output shape is smaller than the input, which 
        requires a skip connection layer size to be matched with current layer. 
        In this application, since our input and output are to be same size, we will note crop the 
        skip connection layer. However, there is a placehold commented, if needed in future. 
        
        '''
        x = self.upsample(x)
        x = self.conv1(x)        
        x = torch.cat([x, skip_con_x], axis=1)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.conv3(x)
        x = self.activation(x)
        return x

In [ ]:
class FeatureMapBlock(nn.Module):
    '''
    FeatureMapBlock Class
    The final layer of a UNet - 
    maps each pixel to a pixel with the correct number of output dimensions
    using a 1x1 convolution.
    Values:
        input_channels: the number of channels to expect from a given input
    '''
    def __init__(self, input_channels, output_channels):
        super(FeatureMapBlock, self).__init__()
        
        # "Every step in the expanding path consists of an upsampling of the feature map"
        self.conv = nn.Conv2d(input_channels, output_channels, kernel_size=1)
        

    def forward(self, x):
        '''
        Function for completing a forward pass of FeatureMapBlock: 
        Given an image tensor, returns it mapped to the desired number of channels.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        x = self.conv(x)
        return x

In [ ]:
class UNet(nn.Module):
    '''
    UNet Class
    A series of 4 contracting blocks followed by 4 expanding blocks to 
    transform an input image into the corresponding paired image, with an upfeature
    layer at the start and a downfeature layer at the end
    Values:
        input_channels: the number of channels to expect from a given input
        output_channels: the number of channels to expect for a given output
    '''
    def __init__(self, input_channels, output_channels, hidden_channels=64):
        super(UNet, self).__init__()        
        # "Every step in the expanding path consists of an upsampling of the feature map"
        self.upfeature = FeatureMapBlock(input_channels, hidden_channels)
        self.contract1 = ContractingBlock(hidden_channels)
        self.contract2 = ContractingBlock(hidden_channels * 2)
        self.contract3 = ContractingBlock(hidden_channels * 4)
        self.contract4 = ContractingBlock(hidden_channels * 8)
        self.expand1 = ExpandingBlock(hidden_channels * 16)
        self.expand2 = ExpandingBlock(hidden_channels * 8)
        self.expand3 = ExpandingBlock(hidden_channels * 4)
        self.expand4 = ExpandingBlock(hidden_channels * 2)
        self.downfeature = FeatureMapBlock(hidden_channels, output_channels)
        
    def forward(self, x):
        '''
        Function for completing a forward pass of UNet: 
        Given an image tensor, passes it through U-Net and returns the output.
        Parameters:
            x: image tensor of shape (batch size, channels, height, width)
        '''
        # Keep in mind that the expand function takes two inputs, 
        # both with the same number of channels.                 
        x0 = self.upfeature(x)
        x1 = self.contract1(x0)        
        x2 = self.contract2(x1)        
        x3 = self.contract3(x2)
        x4 = self.contract4(x3)
        
        x5 = self.expand1(x4, x3)
        x6 = self.expand2(x5, x2)
        x7 = self.expand3(x6, x1)
        x8 = self.expand4(x7, x0)
        xn = self.downfeature(x8)          
        return xn

In [ ]:
def pad_to(image, new_shape):
    '''
    Function for padding an image tensor. 
    If somehow the expanding layer output and the skip connection doesn't match,
    these might be helpful.
    '''
    h, w = image.shape[0], image.shape[1]
    new_h, new_w = new_shape[0], new_shape[1]
    
    inc_h, inc_w = new_h -h, new_w - w
    left, right = 0, inc_w
    top, bottom = 0, inc_h
    pads = left, right, top, bottom 
    
    # zero-padding by default.
    # See others at https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.pad
    padded_image = F.pad(image, pads, "constant", 0)

    return padded_image

In [ ]:
# Special function to display images side by side after training

def show_tensor_images(image, fault, pred, num_images=25, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    # image_shifted = (image_tensor + 1) / 2
    #image_shifted = image_tensor
    
    image_unflat = image.detach().cpu()
    fault_unflat = fault.detach().cpu()
    pred_unflat = pred.detach().cpu()
    
    
    fig = plt.figure(figsize=(12,15))
    
    ax = fig.add_subplot(331)    
    ax.imshow(image_unflat.squeeze(), cmap = 'gray')
    ax.set_title("Seismic Image")
    
    ax = fig.add_subplot(332)
    ax.imshow(image_unflat.squeeze(), cmap = 'gray')
    ax.imshow(fault_unflat.squeeze(), cmap = 'gray', vmin=0, vmax=1, alpha=0.4)
    ax.set_title("Fault")
    
    ax = fig.add_subplot(333)
    ax.imshow(image_unflat.squeeze(), cmap = 'gray')
    ax.imshow(pred_unflat.squeeze(), cmap = 'gray', vmin=0, vmax=1, alpha=0.4)
    ax.set_title("Predicted Fault")
    
    plt.show()

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(device)

In [ ]:
# Hyperparameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 3    
input_dim = 1
label_dim = 1
display_step = 1397
batch_size = 1
lr = 0.0005
initial_shape = 512
target_shape = 512
device = 'cuda'

In [ ]:
def train_valid():
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True)
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size = batch_size,
        shuffle=False)
    unet = UNet(input_dim, label_dim).to(device)
    unet_opt = torch.optim.Adam(unet.parameters(), lr=lr)
    cur_step = 0
    
    train_losses = []
    valid_losses = []

    for epoch in range(n_epochs):
        loss_iter = []
                
        for real, labels in tqdm(train_dataloader):
            
            cur_batch_size = len(real)
            # Flatten the image
            real = real.to(device)
            labels = labels.to(device)            
                      

            ### Update U-Net ###
            unet_opt.zero_grad()
            pred = unet(real)
            #print(pred.shape)
            unet_loss = criterion(pred, labels)
            unet_loss.backward()
            unet_opt.step()
            
            loss_iter.append(unet_loss.item())
            

            if cur_step % display_step == 0:
                print(f"Epoch {epoch}: Step {cur_step}: Training loss: {unet_loss.item()}")
                #show_tensor_images(
                    #crop(real, torch.Size([len(real), 1, target_shape, target_shape])), 
                    #size=(input_dim, target_shape, target_shape)
                #)
                #show_tensor_images(real.T, size=(input_dim, target_shape, target_shape))                
                #show_tensor_images(labels.T, size=(label_dim, target_shape, target_shape))
                #show_tensor_images(torch.sigmoid(pred).T, size=(label_dim, target_shape, target_shape))
                print('Training:')
                show_tensor_images(real.T, labels.T, torch.sigmoid(pred).T, size=(input_dim, target_shape, target_shape))
            cur_step += 1
        
        train_losses.append(statistics.mean(loss_iter))
        print(f"Training loss in all completed epoch: {train_losses}")
        
        with torch.no_grad():
            for val_input, val_labels in tqdm(valid_dataloader):
                val_input = val_input.to(device)
                val_labels = val_labels.to(device)
                val_outputs = unet(val_input)
                val_loss = criterion(val_outputs, val_labels)
        print('Validation:')
        print(f"Validation loss in {epoch} epoch: {val_loss.item()}")
        show_tensor_images(val_input.T, val_labels.T, torch.sigmoid(val_outputs).T, size=(input_dim, target_shape, target_shape))
        
        valid_losses.append(val_loss)       
        
    return unet, pred, train_losses, valid_losses

In [ ]:
model, pred, train_loss, valid_loss = train_valid()

In [ ]:
train_loss_T = torch.Tensor(train_loss)
valid_loss_T = torch.Tensor(valid_loss)
train_loss_cpu = train_loss_T.cpu().detach()
valid_loss_cpu = valid_loss_T.cpu().detach()
fig, ax = plt.subplots(1, figsize=(8,6))
ax.plot(train_loss_cpu, color = 'red', label = 'training loss')
ax.plot(valid_loss_cpu, color = 'blue', label = 'validation loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(range(0,19))
plt.legend(loc='upper right', frameon=False)
plt.show()